###### References: 
- https://docs.python.org/3/library/functions.html   
- Fluent Python by Luciano Ramalho. Chapter 6: Design Patterns with First-Class Functions

# Strategy pattern
    Classic pattern : Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it.
    
![strategy](strategy.png)

### Context
Provides a service by delegating some computation to interchangeable components that implement alternative algorithms. 
### Strategy
The interface common to the components that implement the different algorithms. 
### Concrete Strategy
One of the concrete subclassses of Strategy.

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

In [2]:
Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):  # the Strategy: an Abstract Base Class

    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [3]:
joe = Customer('John Doe', 0)  
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), 
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [4]:
Order(joe, cart, FidelityPromo() )

<Order total: 42.00 due: 42.00>

In [5]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [6]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [7]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [8]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

In [9]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [10]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

The classic example above shows:
- Each concrete strategy as a class with a single method, `discount`. 
- The strategy instances have no state

## Function-Oriented Strategy


In [11]:
from collections import namedtuple

In [12]:
Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)  # To compute discount
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# No abstract class

def fidelity_promo(order):  # Each strategy is a function
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

### Tests

In [13]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [14]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [15]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [16]:
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [17]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

`Strategy objects often make good flyweights.`

A flyweight is a shared object that can be used in multiple contexts simultaneously.

### selecting the best available discount:

In [18]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]  # lists of strategies

def best_promo(order):  # takes an instance of Order, same as promos
    """Select best discount available
    """
    return max(promo(order) for promo in promos)  # Using generator expression

In [19]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [20]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [21]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

### Problem
To add a new promotion strategy, we need to:
- code the function
- add it to the `promos` list

## Finding Strategies in a Module
`global()` Return the dictionary implementing the current module namespace. For code within functions, this is set when the function is defined and remains the same regardless of where the function is called.

In [22]:
promos = [globals()[name] for name in globals()  # Iterate over each name in the dict
            if name.endswith('_promo')
            and name != 'best_promo'] # Filter out best_promo to avoid infinite recursion

Create a module and put all the strategy functions there. I.e. `promotions.py`

In [23]:
import inspect

import promotions

In [24]:
promos = [func for name, func in
                inspect.getmembers(promotions, inspect.isfunction)]

# Command Pattern
![command](command.png)

To decouple an object that invokes an operation (the Invoker) from the provider object that implements it (the Reciever).

I.e. put a `Command` object between two objects. Implementing an interface with a single method, `execute`.

`"Commands are an object-oriented replacement for callbacks."`

In [25]:
class MacroCommand:
    """A command that executes a list of commands"""
    def __init__(self, commands):
        self.commands = list(commands)
    
    def __call__(self):
        for command in self.commands:
            command()

#### Ref: 
"Recipe 8.21. Implementing the Visitor Pattern," in the _Python Cookbook, 3rd Edition_ by David Beazley and Brian K. Jones

_Learning Python Design Pattern_ by Gennaidiy Zlobin

"Useful Design Patterns," in _Expert Python Programming_ by Tarek Ziade